# Compare N2 decadal change in the global ocean using EasyOcean data

Preprocess:
1. Use configurated data from https://cchdo.ucsd.edu/products/goship-easyocean
2. Use gridded data (.mat) with stratification calculations (STrun.m; updated grid_data_pressure.m), note that data must be preprocessed to pressure resolution 2dbar


## Update:
- 2022-03-24 S.Tan: __[fix bug]__ adapted from Zhang et al (2021): If the isotherm is segmented by topography, each portion is assumed to be statistically independent and to contribute at least one DOF. trend_in_multiple_blocks(), trend_in_multiple_z_blocks, and trend_in_multiple_sig_blocks are all modified.
- 2022-03-30 S.Tan: problematic - A01 (.mat), should I blend it with 75n? What's wrong with SR01? Why P15 and P17, I06S and I07, I01 and I02 lack of data?
- 2022-07-06 S.Tan: ploted the number of measurements of each EasyOcean sections (code: N2change_EasyOcean_sections.ipynb) and found out many sections are lack of repeative measurements. Only bug was I06S and AR07E, somehow lack of data.
- 2022-07-07 S.Tan: fixed a bug in trend_in_blocks (tim_sec will always be ~nan, the previous bug was when time[0] is nan, all tim_sec will be nan and ruin the linear regression). Now solve yesterday's bug. 
- 2022-07-07 S.Tan: why a01 (.mat) does not have data?
- 2022-08-22 S.Tan: update the EasyOcean data as of Sept 2022, A01 incorporate to AR07, A20, A22 add 2021 occupations
- __2022-11-22 S.Tan: incorporate the vertical decorrelation length scale 80 m in the estimatre of DOF, the 80 m comes from twice the mean of the maximum of the integrated trend autocorrelation__

In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import scipy.io as sio
import os
import matplotlib.pyplot as plt

import utils as utils

## 1. Code

### 1). Load data

In [2]:
class load_data:
    """
    read data EasyOcean from GOSHIP-EasyOcean gridded data
    """
    def data_EasyOcean_grid(data0):
        """read gridded data from .mat and output xarray.Dataset"""
        NTime = np.zeros((len(data0['ll_grid']), len(data0['D_pr'])), dtype='datetime64[ns]')
        CTDCT = np.zeros((len(data0['pr_grid']), len(data0['ll_grid']), len(data0['D_pr']))) + np.nan
        CTDSA = np.zeros(CTDCT.shape) + np.nan
        CTDsig4 = np.zeros(CTDCT.shape) + np.nan
        CTDN2 = np.zeros(CTDCT.shape) + np.nan
        CTDN2_f1 = np.zeros(CTDCT.shape) + np.nan
        CTDN2_f2 = np.zeros(CTDCT.shape) + np.nan
        CTDN2_f3 = np.zeros((len(data0['pr_grid_lr']), len(data0['ll_grid']), len(data0['D_pr']))) + np.nan
        CTDN2_f4 = np.zeros(CTDN2_f3.shape) + np.nan
    #     CTDtem = np.zeros(CTDCT.shape) + np.nan
    #     CTDsal = np.zeros(CTDCT.shape) + np.nan
    #     CTDoxy = np.zeros(CTDCT.shape) + np.nan

        for i in range(len(data0['D_pr'])):
            NTime[:, i] = utils.datenum_to_datetime64(data0['D_pr'][i].NTime,0)
            CTDCT[:, :, i] = data0['D_pr'][i].CTDCT
            CTDSA[:, :, i] = data0['D_pr'][i].CTDSA
            CTDsig4[:, :, i] = data0['D_pr'][i].CTDsig4
            CTDN2[:, :, i] = data0['D_pr'][i].CTDN2
            CTDN2_f1[:, :, i] = data0['D_pr'][i].CTDN2_f1
            CTDN2_f2[:, :, i] = data0['D_pr'][i].CTDN2_f2
            CTDN2_f3[:, :, i] = data0['D_pr'][i].CTDN2_f3
            CTDN2_f4[:, :, i] = data0['D_pr'][i].CTDN2_f4
    #         CTDtem[:,:,i] = data0['D_pr'][i].CTDtem
    #         CTDsal[:,:,i] = data0['D_pr'][i].CTDsal
    #         CTDoxy[:,:,i] = data0['D_pr'][i].CTDoxy

        # build dataset
        section_time = xr.DataArray(NTime, dims=['latitude_longitude', 'gridded_section'],
                                    coords={'latitude_longitude': data0['ll_grid'],
                                            'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                    attrs={'long_name': 'section_time'})
        absolute_salinity = xr.DataArray(CTDSA, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                                         coords={'latitude_longitude': data0['ll_grid'],
                                                 'pressure': data0['pr_grid'],
                                                 'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                         attrs={'long_name': 'absolute_salinity'})
        conservative_temperature = xr.DataArray(CTDCT, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                                                coords={'latitude_longitude': data0['ll_grid'],
                                                        'pressure': data0['pr_grid'],
                                                        'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                                attrs={'long_name': 'conservative_temperature'})
        sig4 = xr.DataArray(CTDsig4, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                            coords={'latitude_longitude': data0['ll_grid'],
                                    'pressure': data0['pr_grid'],
                                    'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                            attrs={'long_name': 'sig4'})
        nsquares = xr.DataArray(CTDN2, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                                coords={'latitude_longitude': data0['ll_grid'],
                                        'pressure': data0['pr_grid'],
                                        'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                attrs={'long_name': 'nsquares'})
        nsquares_filter1 = xr.DataArray(CTDN2_f1, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                                        coords={'latitude_longitude': data0['ll_grid'],
                                                'pressure': data0['pr_grid'],
                                                'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                        attrs={'long_name': 'nsquares from filtered CT & SA'})
        nsquares_filter2 = xr.DataArray(CTDN2_f2, dims=['pressure', 'latitude_longitude', 'gridded_section'],
                                        coords={'latitude_longitude': data0['ll_grid'],
                                                'pressure': data0['pr_grid'],
                                                'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                        attrs={'long_name': 'nsquares from filtered sig4'})
        nsquares_filter3 = xr.DataArray(CTDN2_f3, dims=['pressure_lr', 'latitude_longitude', 'gridded_section'],
                                        coords={'latitude_longitude': data0['ll_grid'],
                                                'pressure_lr': data0['pr_grid_lr'],
                                                'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                        attrs={'long_name': 'nsquares from linear least-square fit in sig4 layers'})
        nsquares_filter4 = xr.DataArray(CTDN2_f4, dims=['pressure_lr', 'latitude_longitude', 'gridded_section'],
                                        coords={'latitude_longitude': data0['ll_grid'],
                                                'pressure_lr': data0['pr_grid_lr'],
                                                'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                                        attrs={'long_name': 'nsquares from linear least-square fit in local pd layers'})

        Data = xr.Dataset({'section_time': (['latitude_longitude', 'gridded_section'], section_time),
                           'absolute_salinity': (['pressure', 'latitude_longitude', 'gridded_section'], absolute_salinity),
                           'conservative_temperature': (['pressure', 'latitude_longitude', 'gridded_section'], conservative_temperature),
                           'sig4': (['pressure', 'latitude_longitude', 'gridded_section'], sig4),
                           'nsquares': (['pressure', 'latitude_longitude', 'gridded_section'], nsquares),
                           'nsquares_filter1': (['pressure', 'latitude_longitude', 'gridded_section'], nsquares_filter1),
                           'nsquares_filter2': (['pressure', 'latitude_longitude', 'gridded_section'], nsquares_filter2),
                           'nsquares_filter3': (['pressure_lr', 'latitude_longitude', 'gridded_section'], nsquares_filter3),
                           'nsquares_filter4': (['pressure_lr', 'latitude_longitude', 'gridded_section'], nsquares_filter4)},
                          coords={'latitude_longitude': data0['ll_grid'],
                                  'pressure': data0['pr_grid'],
                                  'pressure_lr': data0['pr_grid_lr'],
                                  'gridded_section': np.arange(1, len(data0['D_pr'])+1, 1, int)},
                          attrs={'title': 'Gridded data',
                                 'description': '1. compute N2 and N2_filter from reported data 2. vertical linear interpolation 3. lateral linear interpolation '})

        return Data

In [3]:
def convert_data(
    data, 
):
    """
    output Data from data, picking out N2 computed using desired method
    

    Parameters
    ------
    data : XarrayDataset
           in-situ section

    Returns
    -------
    Data : processed data, XarrayDataset, N2 from sig4 regression
    """  
    z_span = np.diff(data.pressure_lr.data)[0]/2
    z_center = data.pressure_lr.data 
    
    
    SA = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure.data))) + np.nan
    CT = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure.data))) + np.nan
    Sig4 = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure.data))) + np.nan
    N2 = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure_lr.data))) + np.nan
    Sig4_N2 = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure_lr.data))) + np.nan
    CT_N2 = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure_lr.data))) + np.nan
    SA_N2 = np.zeros((len(data.gridded_section.data), len(data.latitude_longitude.data), len(data.pressure_lr.data))) + np.nan
    
    for i in range(len(data.gridded_section.data)):
        for j in range(len(data.latitude_longitude.data)):
            SA[i, j, :] = data.absolute_salinity.data[:, j, i].copy()
            CT[i, j, :] = data.conservative_temperature.data[:, j, i].copy()
            Sig4[i, j, :] = data.sig4.data[:, j, i].copy()
            
            N2[i, j, :] = data.nsquares_filter4.data[:, j, i].copy()
            sig4_blocks, z_blocks, var_blocks_idx = utils.get_blocks(
                Sig4[i, j, :], data.pressure.data, z_span, z_center)       
            ct_blocks = utils.get_blocks(CT[i, j, :], data.pressure.data, z_span, z_center)[0]
            sa_blocks = utils.get_blocks(SA[i, j, :], data.pressure.data, z_span, z_center)[0]
            for k in range(len(z_center)):
                Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
                CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
                SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
      

    Data = xr.Dataset({'SA': (['N_Occup', 'LL', 'Pressure'], SA),
                       'CT': (['N_Occup', 'LL', 'Pressure'],  CT),
                       'Sig4': (['N_Occup', 'LL', 'Pressure'], Sig4),
                       'N2': (['N_Occup', 'LL', 'z_N2'],  N2),
                       'Sig4_N2': (['N_Occup', 'LL', 'z_N2'],  Sig4_N2),
                       'CT_N2': (['N_Occup', 'LL', 'z_N2'],  CT_N2),
                       'SA_N2': (['N_Occup', 'LL', 'z_N2'],  SA_N2),
                       'time': (['N_Occup', 'LL'],  data.section_time.data.T)},
                      coords={'Pressure': data.pressure.data,
                              'N_Occup': data.gridded_section.data,
                              'LL': data.latitude_longitude.data,
                              'z_N2': data.pressure_lr.data},
                      attrs={'title': 'converted EasyOcean gridded data, z_N2 is pressure'})
    
    return Data


### 2). Compute trend

In [4]:
class significant_test:
    def effective_degree_of_freedom(dis, L_decorr = 163e+3):
        """horizontal distance/ decorrelation length scale from Purkey and Johnson 2010"""
        return dis/L_decorr

    

    def effective_degree_of_freedom_v(dis_v, L_decorr_v = 80):
        """vertical distance/ decorrelation length scale from the mode of integrated N2 trend autocorrelation"""
        return dis_v/L_decorr_v

    
    
    def confidence_interval_student_t(sample_mean, sample_std, degrees_freedom, confidence_level=0.95):
        """confidence level by default is two-sided in scipy"""

        import scipy
        sample_standard_error = sample_std/np.sqrt(degrees_freedom)
        CI = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
        standard_error = sample_standard_error.copy()
            
        return CI, standard_error


In [5]:
class trend_in_blocks:
    """compute N2 trend, and confidence interval in spacial blocks"""
    
    
    def trend_n_yr(var, time, n):
        """ trend for 1d or 2d numpy array [time, depth]
        time in timedelta64[ns]
        need to add significant test feature"""

        from scipy import stats
        if len(time[~np.isnan(time)])>0:
            tim_sec = (time - time[~np.isnan(time)][0]).astype('timedelta64[s]').astype('float')
        else:
            tim_sec = (time - time[0]).astype('timedelta64[s]').astype('float')

        if len(var.shape) == 1:
            bad_indexes = np.isnan(var)
            good_indexes = np.logical_not(bad_indexes)
            if len(var[good_indexes])>1:
                res = stats.linregress(tim_sec[good_indexes], var[good_indexes])
                trend_nyr = res.slope*utils.sec_in_yr()*n  
            else:
                trend_nyr = np.nan

        elif len(var.shape) == 2:

            trend_nyr = np.zeros(var.shape[1], ) + np.nan
            for i in range(var.shape[1]):
                bad_indexes = np.isnan(var[:, i])
                good_indexes = np.logical_not(bad_indexes)
                if len(var[good_indexes, i])>1:
                    res = stats.linregress(tim_sec[good_indexes], var[good_indexes, i])
                    trend_nyr[i] = res.slope*utils.sec_in_yr()*n       
        else:
            raise ValueError('var must be 1d or 2d!')

        return trend_nyr
    
    
    

    def trend_in_multiple_z_blocks(Data, param_region, nyr, L_1degree=11e+4, confidence_level=0.95):
        """trend in multiple blocks, 
        starting from param_region["ll_st"], devided by param_region["dl"];
        also average trend within z blocks, 
        return N trend in nyr (float), confidence intervals estimtaed from decorrelation length = 160 km following Purkey and Johnson (2010)
        """

        # horizontal blocks
        if len(param_region["ll_st"]) > 0:
            ll_st = param_region["ll_st"]
        else:
            ll_st = Data.LL.data
        dl = np.abs(np.diff(ll_st)[0])

        # vertical blocks
        if len(param_region["z_center"]) > 0:
            z_N2 = param_region["z_center"]
        else:
            z_N2 = Data.z_N2.data
        dz = np.abs(np.diff(z_N2)[0])

        L_interval = np.abs(np.diff(Data.LL.data)[0])
        N_Occup = Data.N_Occup.data

        trend_N2_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_factor_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_factor_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
        trend_CT_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_CT_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_percentage_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_percentage_err = np.zeros((len(ll_st), len(z_N2))) + np.nan  
        
        time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
        time_mean.fill(np.datetime64("NaT"))
        # for student test
        edof_blk = np.zeros((len(ll_st), len(z_N2)))
        CI_trend_N2 = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_N2_factor = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_N2_percentage = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_CT = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan

        count_seg = 0
        for ll_st_ in ll_st:
            ll_ed_ = ll_st_+dl
            mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

            for j in range(len(N_Occup)):
                if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
                    time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

            trend_N2 = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
            trend_N2_factor = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
            trend_N2_percentage = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
            trend_CT = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
            trend_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_N2_factor_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_N2_percentage_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_CT_blocks = np.zeros((10000, len(z_N2))) + np.nan
            z_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
            ll_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan

            # point to point trend within block
            count = np.zeros(len(z_N2), dtype='int')
            for i in range(len(Data.LL[mask])):
                trend_N2[i,:] = trend_in_blocks.trend_n_yr(Data.N2[:,mask,:].data[:,i,:], Data.time[:,mask].data[:,i], nyr)
                trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
                trend_N2_percentage[i,:] = trend_N2[i,:]/Data.N2[0,mask,:].data[i,:]
                trend_CT[i,:] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,mask,:].data[:,i,:], Data.time[:,mask].data[:,i], nyr)
                trend_N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(trend_N2[i,:], Data.z_N2.data, dz/2, z_N2)
                trend_N2_factor_blocks_, _, _ = utils.get_blocks(trend_N2_factor[i,:], Data.z_N2.data, dz/2, z_N2)
                trend_N2_percentage_blocks_, _, _ = utils.get_blocks(trend_N2_percentage[i,:], Data.z_N2.data, dz/2, z_N2)
                trend_CT_blocks_, _, _ = utils.get_blocks(trend_CT[i,:], Data.z_N2.data, dz/2, z_N2)
                for k in range(len(z_N2)):
                    trend_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = trend_N2_blocks_[k]
                    trend_N2_factor_blocks[count[k]:count[k]+len(trend_N2_factor_blocks_[k]), k] = trend_N2_factor_blocks_[k]
                    trend_N2_percentage_blocks[count[k]:count[k]+len(trend_N2_percentage_blocks_[k]), k] = trend_N2_percentage_blocks_[k]
                    trend_CT_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = trend_CT_blocks_[k]
                    z_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = z_N2_blocks_[k]
                    ll_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = Data.LL[mask].data[i]
                    count[k] += len(trend_N2_blocks_[k])

            # mean trend within block
            for k in range(len(z_N2)):
                if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
                    trend_N2_mean[count_seg, k] = np.nanmean(trend_N2_blocks[:,k])
                if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
                    trend_N2_factor_mean[count_seg, k] = np.nanmean(trend_N2_factor_blocks[:,k])                    
                if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
                    trend_N2_percentage_mean[count_seg, k] = np.nanmean(trend_N2_percentage_blocks[:,k])                    
                if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
                    trend_CT_mean[count_seg, k] = np.nanmean(trend_CT_blocks[:,k])

            # compute effective dof within block
            # dof account for vertical deoccorelation length scale
            for k in range(len(z_N2)):
                for kk in range(len(z_N2_blocks_[k])):
                    if trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].shape[0]>1:
                        var = trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].squeeze()
                    else:
                        var = trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]]
                    _, var_st, var_ed = utils.get_nans_blocks_length(var, False)

                    if len(var_st) > 1:
                        for i in range(len(var_st)):
                            dis_ = (len(var[var_st[i]:var_ed[i]+1])-1) * L_interval * L_1degree
                            edof_ = significant_test.effective_degree_of_freedom(dis_)
                            if (significant_test.effective_degree_of_freedom(dl * L_1degree) > 1) & (edof_ < 1):
                                edof_ = 1
                            edof_blk[count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0]) #*significant_test.effective_degree_of_freedom_v(z_N2_blocks_[k][-1]-z_N2_blocks_[k][0])
                    elif len(var_st) == 1:
                        dis_ = (len(var[var_st[0]:var_ed[0]+1])-1) * L_interval * L_1degree
                        edof_ = significant_test.effective_degree_of_freedom(dis_)
                        edof_blk[count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0]) #*significant_test.effective_degree_of_freedom_v(z_N2_blocks_[k][-1]-z_N2_blocks_[k][0])

            
            # significant test of mean trend within block
            for k in range(len(z_N2)):
                if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
                    CI_trend_N2[count_seg,k,:], trend_N2_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_blocks[:,k]), np.nanstd(trend_N2_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
                    CI_trend_N2_factor[count_seg,k,:], trend_N2_factor_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_factor_blocks[:,k]), np.nanstd(trend_N2_factor_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
                    CI_trend_N2_percentage[count_seg,k,:], trend_N2_percentage_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_percentage_blocks[:,k]), np.nanstd(trend_N2_percentage_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
                    CI_trend_CT[count_seg,k,:], trend_CT_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_CT_blocks[:,k]), np.nanstd(trend_CT_blocks[:,k]), edof_blk[count_seg, k], confidence_level)

            count_seg += 1


        data_N2_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2_mean), 
                                  'trend_N2_err': (['LL_st', 'z_N2'], trend_N2_err), 
                                  'trend_N2_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2),
                                  'trend_N2_factor_mean': (['LL_st', 'z_N2'], trend_N2_factor_mean), 
                                  'trend_N2_factor_err': (['LL_st', 'z_N2'], trend_N2_factor_err), 
                                  'trend_N2_factor_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2_factor),
                                  'trend_N2_percentage_mean': (['LL_st', 'z_N2'], trend_N2_percentage_mean), 
                                  'trend_N2_percentage_err': (['LL_st', 'z_N2'], trend_N2_percentage_err), 
                                  'trend_N2_percentage_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2_percentage),
                                  'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_mean), 
                                  'trend_CT_err': (['LL_st', 'z_N2'], trend_CT_err), 
                                  'trend_CT_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_CT),
                                  'time_mean': (['N_Occup', 'LL_st'], time_mean),
                                  'eDOF': (['LL_st', 'z_N2'], edof_blk),},
                      coords={'N_Occup': N_Occup,
                              'LL_st': ll_st,
                              'z_N2': z_N2,
                              'two-sided': np.array([-1,1])},
                      attrs={'title': 'N2 data in blocks, given occupations'})      


        return data_N2_trend

    
    
    def trend_in_multiple_sig_blocks(Data, param_region, nyr, L_1degree=11e+4, confidence_level=0.95):
        """trend in multiple blocks, 
        starting from param_region["ll_st"], devided by param_region["dl"];
        also average trend within z blocks, 
        return N trend in nyr (float), confidence intervals estimtaed from decorrelation length = 160 km following Purkey and Johnson (2010)
        """

        from scipy.interpolate import interp1d

        # horizontal blocks
        if len(param_region["ll_st"]) > 0:
            ll_st = param_region["ll_st"]
        else:
            ll_st = Data.LL.data
        dl = np.abs(np.diff(ll_st)[0])

        # vertical blocks
        if len(param_region['sig4_center']) > 0:
            z_N2 = param_region['sig4_center']
        else:
            raise ValueError('param_region[''sig4_center''] must not be empty')        
        dz = np.abs(np.diff(z_N2)[0])

        L_interval = np.abs(np.diff(Data.LL.data)[0])
        N_Occup = Data.N_Occup.data

        trend_N2_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_factor_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_factor_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
        trend_N2_percentage_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_N2_percentage_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
        trend_CT_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
        trend_CT_err = np.zeros((len(ll_st), len(z_N2))) + np.nan

        time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
        time_mean.fill(np.datetime64("NaT"))

        # for student test
        CI_trend_N2 = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_N2_factor = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_N2_percentage = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        CI_trend_CT = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
        edof_blk = np.zeros((len(ll_st), len(z_N2)))
        edof_blk_ = np.zeros((len(N_Occup), len(ll_st), len(z_N2)))
        num_blk = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) 
        
        N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        CT_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        z_N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan     
        
        count_seg = 0
        for ll_st_ in ll_st:
            ll_ed_ = ll_st_+dl
            mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

            for j in range(len(N_Occup)):
                if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
                    time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

            # compute EDOF
            # dof account for vertical deoccorelation length scale
            for j in range(len(N_Occup)):
                for i in range(len(Data.LL[mask])):
                    N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                    CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                    for k in range(len(z_N2)):
                        num_blk[j, count_seg, k] += len(z_N2_blocks_[k])
                        N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
                        CT_blocks[j, count_seg, k] = np.nanmean(CT_blocks_[k])
                        z_N2_blocks[j, count_seg, k] = np.nanmean(z_N2_blocks_[k])

            num_ = num_blk[:, count_seg, :].copy()
            num_[num_==0] = np.nan
            edof_blk_[:, count_seg, :] = significant_test.effective_degree_of_freedom((num_-1)* L_interval * L_1degree)*(num_-1)/num_*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])
            edof_blk[count_seg, :] = np.nanmin(edof_blk_[:, count_seg, :], axis=0)

            # point to point trend within block, where data are interpolated onto finer sigma grids
            trend_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_N2_factor_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_N2_percentage_blocks = np.zeros((10000, len(z_N2))) + np.nan
            trend_CT_blocks = np.zeros((10000, len(z_N2))) + np.nan

            count = np.zeros(len(z_N2), dtype='int')
            for k in range(len(z_N2)):
                z_N2_ = np.arange(z_N2[k]-dz/2, z_N2[k]+dz/2+1e-10, dz/5)
                N2_z_N2_ = np.zeros((len(N_Occup), len(z_N2_))) + np.nan
                CT_z_N2_ = np.zeros((len(N_Occup), len(z_N2_))) + np.nan
                for i in range(len(Data.LL[mask])):
                    for j in range(len(N_Occup)):
                        N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                        CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)            
                        if len(z_N2_blocks_[k]) > 1:
                            f = interp1d(z_N2_blocks_[k], N2_blocks_[k], bounds_error=False, fill_value=np.nan)
                            N2_z_N2_[j,:] = f(z_N2_)
                            f = interp1d(z_N2_blocks_[k], CT_blocks_[k], bounds_error=False, fill_value=np.nan)
                            CT_z_N2_[j,:] = f(z_N2_)

                    trend_N2_z_N2_ = trend_in_blocks.trend_n_yr(N2_z_N2_, time_mean[:, count_seg], nyr)
                    trend_N2_factor_z_N2_ = 10**(trend_in_blocks.trend_n_yr(np.log10(N2_z_N2_), time_mean[:, count_seg], nyr))
                    trend_N2_percentage_z_N2_ = trend_N2_z_N2_/N2_z_N2_[0,:]
                    trend_CT_z_N2_ = trend_in_blocks.trend_n_yr(CT_z_N2_, time_mean[:, count_seg], nyr)

                    trend_N2_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_z_N2_
                    trend_N2_factor_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_factor_z_N2_
                    trend_N2_percentage_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_percentage_z_N2_
                    trend_CT_blocks[count[k]:count[k]+len(z_N2_), k] = trend_CT_z_N2_
                    count[k] += len(trend_N2_z_N2_)
                    
            # mean trend within block
            for k in range(len(z_N2)):
                if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
                    trend_N2_mean[count_seg, k] = np.nanmean(trend_N2_blocks[:,k])
                if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
                    trend_N2_factor_mean[count_seg, k] = np.nanmean(trend_N2_factor_blocks[:,k])
                if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
                    trend_N2_percentage_mean[count_seg, k] = np.nanmean(trend_N2_percentage_blocks[:,k])
                if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
                    trend_CT_mean[count_seg, k] = np.nanmean(trend_CT_blocks[:,k])
        
            # significant test of mean trend within block
            for k in range(len(z_N2)):
                if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
                    CI_trend_N2[count_seg,k,:], trend_N2_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_blocks[:,k]), np.nanstd(trend_N2_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
                    CI_trend_N2_factor[count_seg,k,:], trend_N2_factor_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_factor_blocks[:,k]), np.nanstd(trend_N2_factor_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
                    CI_trend_N2_percentage[count_seg,k,:], trend_N2_percentage_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_percentage_blocks[:,k]), np.nanstd(trend_N2_percentage_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
                if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
                    CI_trend_CT[count_seg,k,:], trend_CT_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_CT_blocks[:,k]), np.nanstd(trend_CT_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
            
            count_seg += 1


        data_N2_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2_mean), 
                                  'trend_N2_err': (['LL_st', 'z_N2'], trend_N2_err), 
                                  'trend_N2_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2),
                                  'trend_N2_factor_mean': (['LL_st', 'z_N2'], trend_N2_factor_mean), 
                                  'trend_N2_factor_err': (['LL_st', 'z_N2'], trend_N2_factor_err), 
                                  'trend_N2_factor_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2_factor),  
                                  'trend_N2_percentage_mean': (['LL_st', 'z_N2'], trend_N2_percentage_mean), 
                                  'trend_N2_percentage_err': (['LL_st', 'z_N2'], trend_N2_percentage_err), 
                                  'trend_N2_percentage_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_N2_percentage),                                      
                                  'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_mean), 
                                  'trend_CT_err': (['LL_st', 'z_N2'], trend_CT_err), 
                                  'trend_CT_CI': (['LL_st', 'z_N2', 'two-sided'], CI_trend_CT),
                                  'time_mean': (['N_Occup', 'LL_st'], time_mean),
                                  'eDOF': (['LL_st', 'z_N2'], edof_blk),},
                      coords={'N_Occup': N_Occup,
                              'LL_st': ll_st,
                              'z_N2': z_N2,
                              'two-sided': np.array([-1,1])},
                      attrs={'title': 'N2 data in blocks, given occupations'})      


        return data_N2_trend



In [6]:
class var_in_blocks:
    """compute N2 and CT mean, and confidence interval in spacial blocks"""

    

    def var_in_multiple_z_blocks(Data, param_region, L_1degree=11e+4, confidence_level=0.95):
        """var in multiple blocks, 
        starting from param_region["ll_st"], devided by param_region["dl"];
        also average var within z blocks, 
        return mean, confidence intervals estimtaed from decorrelation length = 160 km following Purkey and Johnson (2010)
        """

        # horizontal blocks
        if len(param_region["ll_st"]) > 0:
            ll_st = param_region["ll_st"]
        else:
            ll_st = Data.LL.data
        dl = np.abs(np.diff(ll_st)[0])

        # vertical blocks
        if len(param_region["z_center"]) > 0:
            z_N2 = param_region["z_center"]
        else:
            z_N2 = Data.z_N2.data
        dz = np.abs(np.diff(z_N2)[0])
        

        L_interval = np.abs(np.diff(Data.LL.data)[0])
        N_Occup = Data.N_Occup.data

        N2_mean = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        N2_err = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        CT_mean = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        CT_err = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan

        time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
        time_mean.fill(np.datetime64("NaT"))
        # for student test
        edof_blk = np.zeros((len(N_Occup), len(ll_st), len(z_N2)))
        CI_N2 = np.zeros((len(N_Occup), len(ll_st), len(z_N2), 2)) + np.nan
        CI_CT = np.zeros((len(N_Occup), len(ll_st), len(z_N2), 2)) + np.nan

        for j in range(len(N_Occup)):
            count_seg = 0
            for ll_st_ in ll_st:
                ll_ed_ = ll_st_+dl
                mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

                if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
                    time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

                N2 = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
                CT = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
                N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
                CT_blocks = np.zeros((10000, len(z_N2))) + np.nan
                z_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
                ll_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan

                # point to point var within block
                count = np.zeros(len(z_N2), dtype='int')
                for i in range(len(Data.LL[mask])):
                    N2[i,:] = Data.N2[j,mask,:].data[i,:]
                    CT[i,:] = Data.CT_N2[j,mask,:].data[i,:]
                    N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(N2[i,:], Data.z_N2.data, dz/2, z_N2)
                    CT_blocks_, _, _ = utils.get_blocks(CT[i,:], Data.z_N2.data, dz/2, z_N2)
                    for k in range(len(z_N2)):
                        N2_blocks[count[k]:count[k]+len(N2_blocks_[k]), k] = N2_blocks_[k]
                        CT_blocks[count[k]:count[k]+len(N2_blocks_[k]), k] = CT_blocks_[k]
                        z_N2_blocks[count[k]:count[k]+len(N2_blocks_[k]), k] = z_N2_blocks_[k]
                        ll_N2_blocks[count[k]:count[k]+len(N2_blocks_[k]), k] = Data.LL[mask].data[i]
                        count[k] += len(N2_blocks_[k])

                # mean var within block
                for k in range(len(z_N2)):
                    if len(N2_blocks[:,k][~np.isnan(N2_blocks[:,k])])>0:
                        N2_mean[j, count_seg, k] = np.nanmean(N2_blocks[:,k])                   
                    if len(CT_blocks[:,k][~np.isnan(CT_blocks[:,k])])>0:
                        CT_mean[j, count_seg, k] = np.nanmean(CT_blocks[:,k])

                # compute effective dof within block
                # dof account for vertical deoccorelation length scale
                for k in range(len(z_N2)):
                    for kk in range(len(z_N2_blocks_[k])):
                        if N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].shape[0]>1:
                            var = N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].squeeze()
                        else:
                            var = N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]]
                        _, var_st, var_ed = utils.get_nans_blocks_length(var, False)

                        if len(var_st) > 1:
                            for i in range(len(var_st)):
                                dis_ = (len(var[var_st[i]:var_ed[i]+1])-1) * L_interval * L_1degree
                                edof_ = significant_test.effective_degree_of_freedom(dis_)
                                if (significant_test.effective_degree_of_freedom(dl * L_1degree) > 1) & (edof_ < 1):
                                    edof_ = 1
                                edof_blk[j, count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])
                        elif len(var_st) == 1:
                            dis_ = (len(var[var_st[0]:var_ed[0]+1])-1) * L_interval * L_1degree
                            edof_ = significant_test.effective_degree_of_freedom(dis_)
                            edof_blk[j, count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])

                # significant test of mean var within block
                for k in range(len(z_N2)):
                    if len(N2_blocks[:,k][~np.isnan(N2_blocks[:,k])])>0:
                        CI_N2[j, count_seg,k,:], N2_err[j, count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(N2_blocks[:,k]), np.nanstd(N2_blocks[:,k]), edof_blk[j, count_seg, k], confidence_level)
                    if len(CT_blocks[:,k][~np.isnan(CT_blocks[:,k])])>0:
                        CI_CT[j, count_seg,k,:], CT_err[j, count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(CT_blocks[:,k]), np.nanstd(CT_blocks[:,k]), edof_blk[j, count_seg, k], confidence_level)

                count_seg += 1


        data_N2 = xr.Dataset({'N2_mean': (['N_Occup', 'LL_st', 'z_N2'], N2_mean), 
                              'N2_err': (['N_Occup', 'LL_st', 'z_N2'], N2_err), 
                              'N2_CI': (['N_Occup', 'LL_st', 'z_N2', 'two-sided'], CI_N2),
                              'CT_mean': (['N_Occup', 'LL_st', 'z_N2'], CT_mean), 
                              'CT_err': (['N_Occup', 'LL_st', 'z_N2'], CT_err), 
                              'CT_CI': (['N_Occup', 'LL_st', 'z_N2', 'two-sided'], CI_CT),
                              'time_mean': (['N_Occup', 'LL_st'], time_mean),
                              'eDOF': (['N_Occup', 'LL_st', 'z_N2'], edof_blk),},
                      coords={'N_Occup': N_Occup,
                              'LL_st': ll_st,
                              'z_N2': z_N2,
                              'two-sided': np.array([-1,1])},
                      attrs={'title': 'N2 data in blocks, given occupations'})      


        return data_N2

    
    
    def var_in_multiple_sig_blocks(Data, param_region, L_1degree=11e+4, confidence_level=0.95):
        """var in multiple blocks, 
        starting from param_region["ll_st"], devided by param_region["dl"];
        also average var within z blocks, 
        return mean, confidence intervals estimtaed from decorrelation length = 160 km following Purkey and Johnson (2010)
        """

        from scipy.interpolate import interp1d

        # horizontal blocks
        if len(param_region["ll_st"]) > 0:
            ll_st = param_region["ll_st"]
        else:
            ll_st = Data.LL.data
        dl = np.abs(np.diff(ll_st)[0])

        # vertical blocks
        if len(param_region['sig4_center']) > 0:
            z_N2 = param_region['sig4_center']
        else:
            raise ValueError('param_region[''sig4_center''] must not be empty')        
        dz = np.abs(np.diff(z_N2)[0])

        L_interval = np.abs(np.diff(Data.LL.data)[0])
        N_Occup = Data.N_Occup.data

        N2_mean = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        N2_err = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan     
        CT_mean = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        CT_err = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan

        time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
        time_mean.fill(np.datetime64("NaT"))

        # for student test
        CI_N2 = np.zeros((len(N_Occup), len(ll_st), len(z_N2), 2)) + np.nan
        CI_CT = np.zeros((len(N_Occup), len(ll_st), len(z_N2), 2)) + np.nan
        edof_blk = np.zeros((len(N_Occup), len(ll_st), len(z_N2)))
        num_blk = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) 
        
        N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        CT_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
        z_N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan     
        
        count_seg = 0
        for ll_st_ in ll_st:
            ll_ed_ = ll_st_+dl
            mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

            for j in range(len(N_Occup)):
                if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
                    time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

            # compute EDOF
            # dof account for vertical deoccorelation length scale
            for j in range(len(N_Occup)):
                for i in range(len(Data.LL[mask])):
                    N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                    CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                    for k in range(len(z_N2)):
                        num_blk[j, count_seg, k] += len(z_N2_blocks_[k])
                        N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
                        CT_blocks[j, count_seg, k] = np.nanmean(CT_blocks_[k])
                        z_N2_blocks[j, count_seg, k] = np.nanmean(z_N2_blocks_[k])

            num_ = num_blk[:, count_seg, :].copy()
            num_[num_==0] = np.nan
            edof_blk[:, count_seg, :] = significant_test.effective_degree_of_freedom((num_-1)* L_interval * L_1degree)*(num_-1)/num_*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])
                   
            # mean trend within block
            N2_mean = N2_blocks.copy()
            CT_mean = CT_blocks.copy()
        
            # significant test of mean trend within block
            for j in range(len(N_Occup)):
                for k in range(len(z_N2)):
                    if len(N2_blocks[j,:,k][~np.isnan(N2_blocks[j,:,k])])>0:
                        CI_N2[j,count_seg,k,:], N2_err[j,count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(N2_blocks[j,:,k]), np.nanstd(N2_blocks[j,:,k]), edof_blk[j, count_seg, k], confidence_level)
                    if len(CT_blocks[j,:,k][~np.isnan(CT_blocks[j,:,k])])>0:
                        CI_CT[j,count_seg,k,:], CT_err[j,count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(CT_blocks[j,:,k]), np.nanstd(CT_blocks[j,:,k]), edof_blk[j, count_seg, k], confidence_level)            
            
            count_seg += 1


        data_N2 = xr.Dataset({'N2_mean': (['N_Occup', 'LL_st', 'z_N2'], N2_mean), 
                              'N2_err': (['N_Occup', 'LL_st', 'z_N2'], N2_err), 
                              'N2_CI': (['N_Occup', 'LL_st', 'z_N2', 'two-sided'], CI_N2),  
                              'CT_mean': (['N_Occup', 'LL_st', 'z_N2'], CT_mean), 
                              'CT_err': (['N_Occup', 'LL_st', 'z_N2'], CT_err), 
                              'CT_CI': (['N_Occup', 'LL_st', 'z_N2', 'two-sided'], CI_CT),
                              'time_mean': (['N_Occup', 'LL_st'], time_mean),
                              'eDOF': (['N_Occup', 'LL_st', 'z_N2'], edof_blk),},
                      coords={'N_Occup': N_Occup,
                              'LL_st': ll_st,
                              'z_N2': z_N2,
                              'two-sided': np.array([-1,1])},
                      attrs={'title': 'N2 data in blocks, given occupations'})      


        return data_N2



## 2. Main Program

### Parameters

In [7]:
dl = 5
dz = 400
dsig = .025
nyr = 10

### 1). Atlantic Ocean

In [8]:
# path to WOCE download data
datafilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/atlantic/'
        
# path to saved data
outfilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'

acquisition_list = ['A05', 'A02', 'A10', 'A20', 'AR07W', 'AR07E', '75N', 'A16-A23', 'A22', 'A13', 'A12']


In [9]:
acquisition_list_ = [ 'A03', ]

In [59]:
Data = xr.open_dataset('/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/sr04.nc')
param_region = {"ll_st": np.arange(Data.LL.data[0], Data.LL.data[-1], dl),
                "z_center": np.arange(Data.z_N2.data[0], Data.z_N2.data[-1], dz),
                "sig4_center": np.arange(45.6, 46.4+1e-4, dsig), 
                "section_name": 'sr04'
            }
L_1degree=11e+4
confidence_level=0.95

# horizontal blocks
if len(param_region["ll_st"]) > 0:
    ll_st = param_region["ll_st"]
else:
    ll_st = Data.LL.data
dl = np.abs(np.diff(ll_st)[0])

# vertical blocks
if len(param_region["z_center"]) > 0:
    z_N2 = param_region["z_center"]
else:
    z_N2 = Data.z_N2.data
dz = np.abs(np.diff(z_N2)[0])

L_interval = np.abs(np.diff(Data.LL.data)[0])
N_Occup = Data.N_Occup.data

trend_N2_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_factor_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_factor_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
trend_CT_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_CT_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_percentage_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_percentage_err = np.zeros((len(ll_st), len(z_N2))) + np.nan  

time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
time_mean.fill(np.datetime64("NaT"))
# for student test
edof_blk = np.zeros((len(ll_st), len(z_N2)))
CI_trend_N2 = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_N2_factor = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_N2_percentage = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_CT = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan

count_seg = 0
for ll_st_ in ll_st:
    ll_ed_ = ll_st_+dl
    mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

    for j in range(len(N_Occup)):
        if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
            time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

    trend_N2 = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
    trend_N2_factor = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
    trend_N2_percentage = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
    trend_CT = np.zeros((len(Data.LL[mask]), len(Data.z_N2))) + np.nan
    trend_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_N2_factor_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_N2_percentage_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_CT_blocks = np.zeros((10000, len(z_N2))) + np.nan
    z_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
    ll_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan

    # point to point trend within block
    count = np.zeros(len(z_N2), dtype='int')
    for i in range(len(Data.LL[mask])):
        trend_N2[i,:] = trend_in_blocks.trend_n_yr(Data.N2[:,mask,:].data[:,i,:], Data.time[:,mask].data[:,i], nyr)
        trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
        trend_N2_percentage[i,:] = trend_N2[i,:]/Data.N2[0,mask,:].data[i,:]
        trend_CT[i,:] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,mask,:].data[:,i,:], Data.time[:,mask].data[:,i], nyr)
        trend_N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(trend_N2[i,:], Data.z_N2.data, dz/2, z_N2)
        trend_N2_factor_blocks_, _, _ = utils.get_blocks(trend_N2_factor[i,:], Data.z_N2.data, dz/2, z_N2)
        trend_N2_percentage_blocks_, _, _ = utils.get_blocks(trend_N2_percentage[i,:], Data.z_N2.data, dz/2, z_N2)
        trend_CT_blocks_, _, _ = utils.get_blocks(trend_CT[i,:], Data.z_N2.data, dz/2, z_N2)
        for k in range(len(z_N2)):
            trend_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = trend_N2_blocks_[k]
            trend_N2_factor_blocks[count[k]:count[k]+len(trend_N2_factor_blocks_[k]), k] = trend_N2_factor_blocks_[k]
            trend_N2_percentage_blocks[count[k]:count[k]+len(trend_N2_percentage_blocks_[k]), k] = trend_N2_percentage_blocks_[k]
            trend_CT_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = trend_CT_blocks_[k]
            z_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = z_N2_blocks_[k]
            ll_N2_blocks[count[k]:count[k]+len(trend_N2_blocks_[k]), k] = Data.LL[mask].data[i]
            count[k] += len(trend_N2_blocks_[k])

    # mean trend within block
    for k in range(len(z_N2)):
        if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
            trend_N2_mean[count_seg, k] = np.nanmean(trend_N2_blocks[:,k])
        if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
            trend_N2_factor_mean[count_seg, k] = np.nanmean(trend_N2_factor_blocks[:,k])                    
        if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
            trend_N2_percentage_mean[count_seg, k] = np.nanmean(trend_N2_percentage_blocks[:,k])                    
        if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
            trend_CT_mean[count_seg, k] = np.nanmean(trend_CT_blocks[:,k])

    # compute effective dof within block
    for k in range(len(z_N2)):
        for kk in range(len(z_N2_blocks_[k])):
            if trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].shape[0]>1:
                var = trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]].squeeze()
            else:
                var = trend_N2[:, Data.z_N2.data == z_N2_blocks_[k][kk]]
            _, var_st, var_ed = utils.get_nans_blocks_length(var, False)

            if len(var_st) > 1:
                for i in range(len(var_st)):
                    dis_ = (len(var[var_st[i]:var_ed[i]+1])-1) * L_interval * L_1degree
                    edof_ = significant_test.effective_degree_of_freedom(dis_)
                    if (significant_test.effective_degree_of_freedom(dl * L_1degree) > 1) & (edof_ < 1):
                        edof_ = 1
                    edof_blk[count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0]) #*significant_test.effective_degree_of_freedom_v(z_N2_blocks_[k][-1]-z_N2_blocks_[k][0])
            elif len(var_st) == 1:
                dis_ = (len(var[var_st[0]:var_ed[0]+1])-1) * L_interval * L_1degree
                edof_ = significant_test.effective_degree_of_freedom(dis_)
                edof_blk[count_seg, k] += edof_*(len(z_N2_blocks_[k])-1)/len(z_N2_blocks_[k])*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0]) #*significant_test.effective_degree_of_freedom_v(z_N2_blocks_[k][-1]-z_N2_blocks_[k][0])
                
    # significant test of mean trend within block
    for k in range(len(z_N2)):
        if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
            CI_trend_N2[count_seg,k,:], trend_N2_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_blocks[:,k]), np.nanstd(trend_N2_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
            CI_trend_N2_factor[count_seg,k,:], trend_N2_factor_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_factor_blocks[:,k]), np.nanstd(trend_N2_factor_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
            CI_trend_N2_percentage[count_seg,k,:], trend_N2_percentage_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_percentage_blocks[:,k]), np.nanstd(trend_N2_percentage_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
            CI_trend_CT[count_seg,k,:], trend_CT_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_CT_blocks[:,k]), np.nanstd(trend_CT_blocks[:,k]), edof_blk[count_seg, k], confidence_level)

    count_seg += 1

/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/3436381732.py:69: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/Users/stan/miniforge3/envs/goship/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2128: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/stan/miniforge3/envs/goship/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2129: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/510401764.py:18: RuntimeWarning: invalid value encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)


In [58]:
significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])

0.5

In [25]:
significant_test.effective_degree_of_freedom_v(dz)

5.0

In [24]:
len(z_N2_blocks_[k])

8

In [23]:
z_N2_blocks_

[array([  0,  40,  80, 120, 160, 200], dtype=uint16),
 array([200, 240, 280, 320, 360, 400, 440, 480, 520, 560, 600],
       dtype=uint16),
 array([ 600,  640,  680,  720,  760,  800,  840,  880,  920,  960, 1000],
       dtype=uint16),
 array([1000, 1040, 1080, 1120, 1160, 1200, 1240, 1280, 1320, 1360, 1400],
       dtype=uint16),
 array([1400, 1440, 1480, 1520, 1560, 1600, 1640, 1680, 1720, 1760, 1800],
       dtype=uint16),
 array([1800, 1840, 1880, 1920, 1960, 2000, 2040, 2080, 2120, 2160, 2200],
       dtype=uint16),
 array([2200, 2240, 2280, 2320, 2360, 2400, 2440, 2480, 2520, 2560, 2600],
       dtype=uint16),
 array([2600, 2640, 2680, 2720, 2760, 2800, 2840, 2880, 2920, 2960, 3000],
       dtype=uint16),
 array([3000, 3040, 3080, 3120, 3160, 3200, 3240, 3280, 3320, 3360, 3400],
       dtype=uint16),
 array([3400, 3440, 3480, 3520, 3560, 3600, 3640, 3680, 3720, 3760, 3800],
       dtype=uint16),
 array([3800, 3840, 3880, 3920, 3960, 4000, 4040, 4080, 4120, 4160, 4200],
       dt

In [22]:
count

array([18, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 24])

In [30]:
z_N2

array([   0,  400,  800, 1200, 1600, 2000, 2400, 2800, 3200, 3600, 4000,
       4400, 4800, 5200, 5600, 6000, 6400])

In [20]:
edof_blk.shape

(10, 17)

In [41]:
edof_blk[:,5], edof_blk[:,0]

(array([ 0.        , 35.49693252, 36.37423313, 36.37423313, 36.37423313,
        36.37423313, 36.37423313, 36.37423313, 30.36809816,  0.        ]),
 array([ 9.24539877, 16.53374233, 16.53374233, 16.53374233, 16.53374233,
        16.53374233, 16.53374233, 16.53374233, 16.53374233,  0.33742331]))

In [60]:
edof_blk[:,5], edof_blk[:,0]

(array([ 0.        , 16.13496933, 16.53374233, 16.53374233, 16.53374233,
        16.53374233, 16.53374233, 16.53374233, 13.80368098,  0.        ]),
 array([3.85224949, 6.8890593 , 6.8890593 , 6.8890593 , 6.8890593 ,
        6.8890593 , 6.8890593 , 6.8890593 , 6.8890593 , 0.14059305]))

In [57]:
5/11, (11-1)*.5/11, 200/80/6, (6-1)*.5/6

(0.45454545454545453,
 0.45454545454545453,
 0.4166666666666667,
 0.4166666666666667)

In [61]:
Data

<xarray.Dataset>
Dimensions:   (LL: 454, N_Occup: 8, Pressure: 651, z_N2: 163)
Coordinates:
  * Pressure  (Pressure) uint16 0 10 20 30 40 50 ... 6460 6470 6480 6490 6500
  * N_Occup   (N_Occup) int64 1 2 3 4 5 6 7 8
  * LL        (LL) float64 303.4 303.5 303.6 303.7 ... 348.4 348.5 348.6 348.7
  * z_N2      (z_N2) uint16 0 40 80 120 160 200 ... 6320 6360 6400 6440 6480
Data variables:
    SA        (N_Occup, LL, Pressure) float64 ...
    CT        (N_Occup, LL, Pressure) float64 ...
    Sig4      (N_Occup, LL, Pressure) float64 ...
    N2        (N_Occup, LL, z_N2) float64 ...
    Sig4_N2   (N_Occup, LL, z_N2) float64 ...
    CT_N2     (N_Occup, LL, z_N2) float64 ...
    SA_N2     (N_Occup, LL, z_N2) float64 ...
    time      (N_Occup, LL) datetime64[ns] NaT ... NaT
Attributes:
    title:    converted EasyOcean gridded data, z_N2 is pressure

In [87]:
num_ 

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5.5, 5. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0.5, 4.5, 5.5, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0.5, 1. , 0.5, 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.

In [91]:
from scipy.interpolate import interp1d

# horizontal blocks
if len(param_region["ll_st"]) > 0:
    ll_st = param_region["ll_st"]
else:
    ll_st = Data.LL.data
dl = np.abs(np.diff(ll_st)[0])

# vertical blocks
if len(param_region['sig4_center']) > 0:
    z_N2 = param_region['sig4_center']
else:
    raise ValueError('param_region[''sig4_center''] must not be empty')        
dz = np.abs(np.diff(z_N2)[0])

L_interval = np.abs(np.diff(Data.LL.data)[0])
N_Occup = Data.N_Occup.data

trend_N2_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_err = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_factor_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_factor_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
trend_N2_percentage_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_N2_percentage_err = np.zeros((len(ll_st), len(z_N2))) + np.nan        
trend_CT_mean = np.zeros((len(ll_st), len(z_N2))) + np.nan
trend_CT_err = np.zeros((len(ll_st), len(z_N2))) + np.nan

time_mean = np.zeros((len(N_Occup), len(ll_st)), dtype='datetime64[ns]')
time_mean.fill(np.datetime64("NaT"))

# for student test
CI_trend_N2 = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_N2_factor = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_N2_percentage = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
CI_trend_CT = np.zeros((len(ll_st), len(z_N2), 2)) + np.nan
edof_blk = np.zeros((len(ll_st), len(z_N2)))
edof_blk_ = np.zeros((len(N_Occup), len(ll_st), len(z_N2)))
num_blk = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) 

N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
CT_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan
z_N2_blocks = np.zeros((len(N_Occup), len(ll_st), len(z_N2))) + np.nan     

count_seg = 0
for ll_st_ in ll_st:
    ll_ed_ = ll_st_+dl
    mask = (Data.LL.data <= ll_ed_) & (Data.LL.data > ll_st_)

    for j in range(len(N_Occup)):
        if len(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])])>0:
            time_mean[j, count_seg] = np.nanmean(Data.time.data[j, mask][~np.isnat(Data.time.data[j,mask])].astype('float')).astype('datetime64[ns]')

    # compute EDOF
    # dof account for vertical deoccorelation length scale
    for j in range(len(N_Occup)):
        for i in range(len(Data.LL[mask])):
            N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
            CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
            for k in range(len(z_N2)):
                num_blk[j, count_seg, k] += len(z_N2_blocks_[k])
                N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
                CT_blocks[j, count_seg, k] = np.nanmean(CT_blocks_[k])
                z_N2_blocks[j, count_seg, k] = np.nanmean(z_N2_blocks_[k])

    num_ = num_blk[:, count_seg, :].copy()
    num_[num_==0] = np.nan
    edof_blk_[:, count_seg, :] = significant_test.effective_degree_of_freedom((num_-1)* L_interval * L_1degree)*(num_-1)/num_*significant_test.effective_degree_of_freedom_v(np.diff(Data.z_N2.data)[0])
    edof_blk[count_seg, :] = np.nanmin(edof_blk_[:, count_seg, :], axis=0)

    # point to point trend within block, where data are interpolated onto finer sigma grids
    trend_N2_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_N2_factor_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_N2_percentage_blocks = np.zeros((10000, len(z_N2))) + np.nan
    trend_CT_blocks = np.zeros((10000, len(z_N2))) + np.nan

    count = np.zeros(len(z_N2), dtype='int')
    for k in range(len(z_N2)):
        z_N2_ = np.arange(z_N2[k]-dz/2, z_N2[k]+dz/2+1e-10, dz/5)
        N2_z_N2_ = np.zeros((len(N_Occup), len(z_N2_))) + np.nan
        CT_z_N2_ = np.zeros((len(N_Occup), len(z_N2_))) + np.nan
        for i in range(len(Data.LL[mask])):
            for j in range(len(N_Occup)):
                N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
                CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)            
                if len(z_N2_blocks_[k]) > 1:
                    f = interp1d(z_N2_blocks_[k], N2_blocks_[k], bounds_error=False, fill_value=np.nan)
                    N2_z_N2_[j,:] = f(z_N2_)
                    f = interp1d(z_N2_blocks_[k], CT_blocks_[k], bounds_error=False, fill_value=np.nan)
                    CT_z_N2_[j,:] = f(z_N2_)

            trend_N2_z_N2_ = trend_in_blocks.trend_n_yr(N2_z_N2_, time_mean[:, count_seg], nyr)
            trend_N2_factor_z_N2_ = 10**(trend_in_blocks.trend_n_yr(np.log10(N2_z_N2_), time_mean[:, count_seg], nyr))
            trend_N2_percentage_z_N2_ = trend_N2_z_N2_/N2_z_N2_[0,:]
            trend_CT_z_N2_ = trend_in_blocks.trend_n_yr(CT_z_N2_, time_mean[:, count_seg], nyr)

            trend_N2_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_z_N2_
            trend_N2_factor_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_factor_z_N2_
            trend_N2_percentage_blocks[count[k]:count[k]+len(z_N2_), k] = trend_N2_percentage_z_N2_
            trend_CT_blocks[count[k]:count[k]+len(z_N2_), k] = trend_CT_z_N2_
            count[k] += len(trend_N2_z_N2_)

    # mean trend within block
    for k in range(len(z_N2)):
        if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
            trend_N2_mean[count_seg, k] = np.nanmean(trend_N2_blocks[:,k])
        if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
            trend_N2_factor_mean[count_seg, k] = np.nanmean(trend_N2_factor_blocks[:,k])
        if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
            trend_N2_percentage_mean[count_seg, k] = np.nanmean(trend_N2_percentage_blocks[:,k])
        if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
            trend_CT_mean[count_seg, k] = np.nanmean(trend_CT_blocks[:,k])

    # significant test of mean trend within block
    for k in range(len(z_N2)):
        if len(trend_N2_blocks[:,k][~np.isnan(trend_N2_blocks[:,k])])>0:
            CI_trend_N2[count_seg,k,:], trend_N2_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_blocks[:,k]), np.nanstd(trend_N2_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_N2_factor_blocks[:,k][~np.isnan(trend_N2_factor_blocks[:,k])])>0:
            CI_trend_N2_factor[count_seg,k,:], trend_N2_factor_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_factor_blocks[:,k]), np.nanstd(trend_N2_factor_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_N2_percentage_blocks[:,k][~np.isnan(trend_N2_percentage_blocks[:,k])])>0:
            CI_trend_N2_percentage[count_seg,k,:], trend_N2_percentage_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_N2_percentage_blocks[:,k]), np.nanstd(trend_N2_percentage_blocks[:,k]), edof_blk[count_seg, k], confidence_level)
        if len(trend_CT_blocks[:,k][~np.isnan(trend_CT_blocks[:,k])])>0:
            CI_trend_CT[count_seg,k,:], trend_CT_err[count_seg,k] = significant_test.confidence_interval_student_t(np.nanmean(trend_CT_blocks[:,k]), np.nanstd(trend_CT_blocks[:,k]), edof_blk[count_seg, k], confidence_level)

    count_seg += 1



/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/1595669061.py:62: RuntimeWarning: Mean of empty slice
  N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/1595669061.py:63: RuntimeWarning: Mean of empty slice
  CT_blocks[j, count_seg, k] = np.nanmean(CT_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/1595669061.py:64: RuntimeWarning: Mean of empty slice
  z_N2_blocks[j, count_seg, k] = np.nanmean(z_N2_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/1595669061.py:69: RuntimeWarning: All-NaN slice encountered
  edof_blk[count_seg, :] = np.nanmin(edof_blk_[:, count_seg, :], axis=0)
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_13606/1595669061.py:93: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor_z_N2_ = 10**(trend_in_blocks.trend_n_yr(np.log10(N2_z_N2_), time_mean[:, count_seg], nyr))
/var/folders/sr/6_bkfhj9293

In [ ]:
# compute EDOF
# dof account for vertical deoccorelation length scale
for j in range(len(N_Occup)):
    for i in range(len(Data.LL[mask])):
        N2_blocks_, z_N2_blocks_, _ = utils.get_blocks(Data.N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
        CT_blocks_, _, _ = utils.get_blocks(Data.CT_N2[:,mask,:].data[j,i,:], Data.Sig4_N2[:,mask,:].data[j,i,:], dz/2, z_N2)
        for k in range(len(z_N2)):
            num_blk[j, count_seg, k] += len(z_N2_blocks_[k])
            N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
            CT_blocks[j, count_seg, k] = np.nanmean(CT_blocks_[k])
            z_N2_blocks[j, count_seg, k] = np.nanmean(z_N2_blocks_[k])

num_ = num_blk[:, count_seg, :].copy()
num_[num_==0] = np.nan
edof_blk_[:, count_seg, :] = significant_test.effective_degree_of_freedom((num_-1)* L_interval * L_1degree)
edof_blk[count_seg, :] = np.nanmin(edof_blk_[:, count_seg, :], axis=0)


In [10]:
for acquisition in acquisition_list: 
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        print(10*"-", acquisition, 10*"-")
        for filename_ in os.listdir(acqui_path):
            if (filename_.endswith(".mat")):  # mat data
                data0 = sio.loadmat(acqui_path + filename_,
                                    squeeze_me=True, struct_as_record=False)
                data = load_data.data_EasyOcean_grid(data0)
                Data = convert_data(data)
                Data.to_netcdf(outfilepath + acquisition.lower() + ".nc")

                param_region = {"ll_st": np.arange(Data.LL.data[0], Data.LL.data[-1], dl),
                                "z_center": np.arange(Data.z_N2.data[0], Data.z_N2.data[-1], dz),
                                "sig4_center": np.arange(45.6, 46.4+1e-4, dsig), 
                                "section_name": acquisition
                            }

                data_N2_trend_z = trend_in_blocks.trend_in_multiple_z_blocks(Data, param_region, nyr)
                data_N2_trend_sig = trend_in_blocks.trend_in_multiple_sig_blocks(Data, param_region, nyr)
                data_N2_trend_z.to_netcdf(outfilepath + acquisition.lower() + "_trend.nc")
                data_N2_trend_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_trend.nc")

                data_N2_z = var_in_blocks.var_in_multiple_z_blocks(Data, param_region)
                data_N2_z.to_netcdf(outfilepath + acquisition.lower() + "_block.nc")
                data_N2_sig = var_in_blocks.var_in_multiple_sig_blocks(Data, param_region)
                data_N2_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_block.nc")
 

---------- A05 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/Users/stan/miniforge3/envs/goship

---------- A02 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- A10 ----------


...

/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- A13 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- A12 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

### 2). Pacific Ocean

In [11]:
# path to WOCE download data
datafilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/pacific/'
        
# path to saved data
outfilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'

acquisition_list = ['P21', 'P17', 'P10', 'P16', 'P18', 'P02', 'P03', 'P13', 'P14', 'P15', 'P01', 'P06', 'P09', 'P17E']

In [12]:
acquisition_list_ = ['P11', 'P04', ]

In [13]:
for acquisition in acquisition_list:
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        print(10*"-", acquisition, 10*"-")
        for filename_ in os.listdir(acqui_path):
            if (filename_.endswith(".mat")):  # mat data
                data0 = sio.loadmat(acqui_path + filename_,
                                    squeeze_me=True, struct_as_record=False)
                data = load_data.data_EasyOcean_grid(data0)
                Data = convert_data(data)
                Data.to_netcdf(outfilepath + acquisition.lower() + ".nc")

                param_region = {"ll_st": np.arange(Data.LL.data[0], Data.LL.data[-1], dl),
                                "z_center": np.arange(Data.z_N2.data[0], Data.z_N2.data[-1], dz),
                                "sig4_center": np.arange(45.6, 46.4+1e-4, dsig),
                                "section_name": acquisition
                            }

                data_N2_trend_z = trend_in_blocks.trend_in_multiple_z_blocks(Data, param_region, nyr)
                data_N2_trend_sig = trend_in_blocks.trend_in_multiple_sig_blocks(Data, param_region, nyr)
                data_N2_trend_z.to_netcdf(outfilepath + acquisition.lower() + "_trend.nc")
                data_N2_trend_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_trend.nc")

                data_N2_z = var_in_blocks.var_in_multiple_z_blocks(Data, param_region)
                data_N2_z.to_netcdf(outfilepath + acquisition.lower() + "_block.nc")
                data_N2_sig = var_in_blocks.var_in_multiple_sig_blocks(Data, param_region)
                data_N2_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_block.nc")


---------- P21 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- P17 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: invalid value encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k5

---------- P10 ----------


...

/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- P09 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- P17E ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

### 3). Indian Ocean

In [14]:
# path to WOCE download data
datafilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/indian/'
        
# path to saved data
outfilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'

acquisition_list = ['I03-I04', 'I08N', 'I08S-I09N', 'I01', 'I07', 'IR06E', 'I06S', 'I02', 'I05', 'I10', 'I09S', 'IR06']

In [15]:
acquisition_list_ = ['IR06-I10',]

In [16]:
for acquisition in acquisition_list: 
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        print(10*"-", acquisition, 10*"-")
        for filename_ in os.listdir(acqui_path):
            if (filename_.endswith(".mat")):  # mat data
                data0 = sio.loadmat(acqui_path + filename_,
                                    squeeze_me=True, struct_as_record=False)
                data = load_data.data_EasyOcean_grid(data0)
                Data = convert_data(data)
                Data.to_netcdf(outfilepath + acquisition.lower() + ".nc")

                param_region = {"ll_st": np.arange(Data.LL.data[0], Data.LL.data[-1], dl),
                                "z_center": np.arange(Data.z_N2.data[0], Data.z_N2.data[-1], dz),
                                "sig4_center": np.arange(45.6, 46.4+1e-4, dsig), 
                                "section_name": acquisition
                            }

                data_N2_trend_z = trend_in_blocks.trend_in_multiple_z_blocks(Data, param_region, nyr)
                data_N2_trend_sig = trend_in_blocks.trend_in_multiple_sig_blocks(Data, param_region, nyr)
                data_N2_trend_z.to_netcdf(outfilepath + acquisition.lower() + "_trend.nc")
                data_N2_trend_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_trend.nc")

                data_N2_z = var_in_blocks.var_in_multiple_z_blocks(Data, param_region)
                data_N2_z.to_netcdf(outfilepath + acquisition.lower() + "_block.nc")
                data_N2_sig = var_in_blocks.var_in_multiple_sig_blocks(Data, param_region)
                data_N2_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_block.nc")
 

---------- I03-I04 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: invalid value encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k5

---------- I08N ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- I08S-I09N ----------


...

/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:265: RuntimeWarning: Mean of empty slice
  N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_240

---------- I09S ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- IR06 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

### 4). Southern Ocean

In [17]:
# path to WOCE download data
datafilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/southern/'
        
# path to saved data
outfilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'

acquisition_list = ['S04I', 'SR03', 'SR04', 'S04P', 'SR01']

In [18]:
for acquisition in acquisition_list: 
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        print(10*"-", acquisition, 10*"-")
        for filename_ in os.listdir(acqui_path):
            if (filename_.endswith(".mat")):  # mat data
                data0 = sio.loadmat(acqui_path + filename_,
                                    squeeze_me=True, struct_as_record=False)
                data = load_data.data_EasyOcean_grid(data0)
                Data = convert_data(data)
                Data.to_netcdf(outfilepath + acquisition.lower() + ".nc")

                param_region = {"ll_st": np.arange(Data.LL.data[0], Data.LL.data[-1], dl),
                                "z_center": np.arange(Data.z_N2.data[0], Data.z_N2.data[-1], dz),
                                "sig4_center": np.arange(45.6, 46.4+1e-4, dsig), 
                                "section_name": acquisition
                            }

                data_N2_trend_z = trend_in_blocks.trend_in_multiple_z_blocks(Data, param_region, nyr)
                data_N2_trend_sig = trend_in_blocks.trend_in_multiple_sig_blocks(Data, param_region, nyr)
                data_N2_trend_z.to_netcdf(outfilepath + acquisition.lower() + "_trend.nc")
                data_N2_trend_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_trend.nc")

                data_N2_z = var_in_blocks.var_in_multiple_z_blocks(Data, param_region)
                data_N2_z.to_netcdf(outfilepath + acquisition.lower() + "_block.nc")
                data_N2_sig = var_in_blocks.var_in_multiple_sig_blocks(Data, param_region)
                data_N2_sig.to_netcdf(outfilepath + acquisition.lower() + "_sig4_block.nc")


---------- S04I ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- SR03 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:265: RuntimeWarning: Mean of empty slice
  N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_240

---------- SR04 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/Users/stan/miniforge3/envs/goship/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2128: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/stan/miniforge3/envs/goship/lib/python3

---------- S04P ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/510401764.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  sample_standard_error = sample_std/np.sqrt(degrees_freedom)
/var/folders/sr/6_bkfhj929364hv72k

---------- SR01 ----------


/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:41: RuntimeWarning: Mean of empty slice
  Sig4_N2[i, j, k] = np.nanmean(sig4_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:42: RuntimeWarning: Mean of empty slice
  CT_N2[i, j, k] = np.nanmean(ct_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/3885223226.py:43: RuntimeWarning: Mean of empty slice
  SA_N2[i, j, k] = np.nanmean(sa_blocks[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:108: RuntimeWarning: invalid value encountered in log10
  trend_N2_factor[i,:] = 10**(trend_in_blocks.trend_n_yr(np.log10(Data.N2[:,mask,:].data[:,i,:]), Data.time[:,mask].data[:,i], nyr))
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_24012/1286468620.py:265: RuntimeWarning: Mean of empty slice
  N2_blocks[j, count_seg, k] = np.nanmean(N2_blocks_[k])
/var/folders/sr/6_bkfhj929364hv72k5pf2jw0000gn/T/ipykernel_240